# 📊 Notebook: Análise de Criadores
Este notebook realiza a análise de métricas de criadores de conteúdo, unindo informações da Wikipedia, posts e usuários do YouTube.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
df_wiki = spark.table("default.creators_scrape_wiki")
df_posts = spark.table("default.posts_creator")
df_users = spark.table("default.users_yt")

In [0]:
df_posts = df_posts.withColumn("published_at", F.from_unixtime("published_at").cast("date"))

In [0]:
max_date = df_posts.select(F.max("published_at")).collect()[0][0]
cutoff_date = F.add_months(F.lit(max_date), -6)

df_posts_6m = df_posts.filter(F.col("published_at") >= cutoff_date)

In [0]:
df_join = df_posts_6m.join(df_users, df_posts_6m.yt_user == df_users.user_id, how="inner")

## 📈 Janelas analíticas
Criação de janelas particionadas por `user_id` para identificar posts com mais likes e mais views.


In [0]:
window_likes = Window.partitionBy("user_id").orderBy(F.desc("likes"))

df_top_likes = (
    df_join
    .withColumn("rank", F.row_number().over(window_likes))
    .filter(F.col("rank") <= 3)
    .select("user_id", "title", "likes", "rank")
)

print("▶️ Top 3 posts por likes (últimos 6 meses):")
df_top_likes.show(truncate=False)

▶️ Top 3 posts por likes (últimos 6 meses):
+-----------------+---------------------------------------------------------------------------------+-------+----+
|user_id          |title                                                                            |likes  |rank|
+-----------------+---------------------------------------------------------------------------------+-------+----+
|CanalKondZilla   |Nike 4 molas ou 12 molas? A história da Nike no Brasil                           |158253 |1   |
|CanalKondZilla   |O de trás tá na maior brisa curtindo um pião com os cara 😂🐶👋                  |93008  |2   |
|CanalKondZilla   |Ondas do Mar - MC Kekel (KondZilla)                                              |73592  |3   |
|MoreZoella       |Introducing Novie & Cosy Family Catch Ups                                        |43533  |1   |
|MoreZoella       |Preparing For The New Baby & Christmas | Last Vlog of 2023                       |39965  |2   |
|MoreZoella       |Autumn Has Arrived |

In [0]:
window_views = Window.partitionBy("user_id").orderBy(F.desc("views"))

df_top_views = (
    df_join
    .withColumn("rank", F.row_number().over(window_views))
    .filter(F.col("rank") <= 3)
    .select("user_id", "title", "views", "rank")
)

print("▶️ Top 3 posts por views (últimos 6 meses):")
df_top_views.show(truncate=False)

▶️ Top 3 posts por views (últimos 6 meses):
+-----------------+---------------------------------------------------------------------------------+--------+----+
|user_id          |title                                                                            |views   |rank|
+-----------------+---------------------------------------------------------------------------------+--------+----+
|CanalKondZilla   |Ondas do Mar - MC Kekel (KondZilla)                                              |7021014 |1   |
|CanalKondZilla   |Nike 4 molas ou 12 molas? A história da Nike no Brasil                           |2089656 |2   |
|CanalKondZilla   |O de trás tá na maior brisa curtindo um pião com os cara 😂🐶👋                  |1959321 |3   |
|MoreZoella       |Preparing For The New Baby & Christmas | Last Vlog of 2023                       |1333391 |1   |
|MoreZoella       |Baking with Ottie, Halloween And House Prep | ad                                 |923701  |2   |
|MoreZoella       |Autumn Has A

## ⚠️ Usuários ausentes
Identificação de usuários que aparecem em posts, mas não estão na base de `users_yt`.


In [0]:
df_missing_users = df_posts.select("yt_user").distinct() \
    .join(df_users.select("user_id").distinct(), df_posts.yt_user == df_users.user_id, how="left_anti")

print("▶️ yt_users presentes em posts_creator mas ausentes em users_yt:")
df_missing_users.show(truncate=False)

▶️ yt_users presentes em posts_creator mas ausentes em users_yt:
+----------------+
|yt_user         |
+----------------+
|tedx Talks      |
|LuisitoComunica |
|checkgate       |
|DottieChicken   |
|tedtalksdirector|
|GallinaPintadita|
|Pirulla25       |
+----------------+



## 📈 Janelas analíticas
Criação de janelas particionadas por `user_id` para identificar posts com mais likes e mais views.


In [0]:
df_posts = df_posts.withColumn("year_month", F.date_format("published_at", "yyyy-MM"))

df_posts_monthly = df_posts.groupBy("yt_user", "year_month") \
    .agg(F.count("*").alias("total_posts")) \
    .orderBy("yt_user", "year_month")

print("▶️ Total de posts por mês por creator:")
df_posts_monthly.show(truncate=False)

▶️ Total de posts por mês por creator:
+----------------+----------+-----------+
|yt_user         |year_month|total_posts|
+----------------+----------+-----------+
|CanalKondZilla  |2023-04   |26         |
|CanalKondZilla  |2023-05   |64         |
|CanalKondZilla  |2023-06   |27         |
|CanalKondZilla  |2023-07   |26         |
|CanalKondZilla  |2023-08   |21         |
|CanalKondZilla  |2023-09   |20         |
|CanalKondZilla  |2023-10   |23         |
|CanalKondZilla  |2023-11   |22         |
|CanalKondZilla  |2023-12   |21         |
|CanalKondZilla  |2024-01   |17         |
|CanalKondZilla  |2024-02   |25         |
|CanalKondZilla  |2024-03   |37         |
|CanalKondZilla  |2024-04   |5          |
|DottieChicken   |2023-10   |1          |
|GallinaPintadita|2023-05   |1          |
|GallinaPintadita|2023-10   |1          |
|LuisitoComunica |2023-04   |5          |
|LuisitoComunica |2023-05   |12         |
|LuisitoComunica |2023-06   |11         |
|LuisitoComunica |2023-07   |9       

## ⚠️ Usuários ausentes
Identificação de usuários que aparecem em posts, mas não estão na base de `users_yt`.


In [0]:
min_date = df_posts.select(F.min("published_at")).first()[0]
max_date = df_posts.select(F.max("published_at")).first()[0]

n_months = int(df_posts.select(
    F.months_between(F.lit(max_date), F.lit(min_date))
).first()[0]) + 1

months_seq = spark.range(0, n_months).withColumn(
    "year_month",
    F.date_format(
        F.add_months(F.lit(min_date), F.col("id")),
        "yyyy-MM"
    )
).select("year_month").distinct()

users = df_posts.select("yt_user").distinct()

users_months = users.crossJoin(months_seq)

df_posts_monthly_full = users_months.join(
    df_posts_monthly,
    on=["yt_user", "year_month"],
    how="left"
).fillna(0, subset=["total_posts"]).orderBy("yt_user", "year_month")

print("▶️ Total de posts por mês por creator (com 0 onde não tem post):")
display(df_posts_monthly_full)

▶️ Total de posts por mês por creator (com 0 onde não tem post):


yt_user,year_month,total_posts
CanalKondZilla,2023-04,26
CanalKondZilla,2023-05,64
CanalKondZilla,2023-06,27
CanalKondZilla,2023-07,26
CanalKondZilla,2023-08,21
CanalKondZilla,2023-09,20
CanalKondZilla,2023-10,23
CanalKondZilla,2023-11,22
CanalKondZilla,2023-12,21
CanalKondZilla,2024-01,17


## 🔄 Pivot mensal de posts
Transforma os dados em formato mensal (séries temporais) para facilitar análises de evolução ao longo do tempo.


In [0]:
df_pivot = df_posts_monthly_full.groupBy("yt_user") \
    .pivot("year_month") \
    .agg(F.first("total_posts")) \
    .fillna(0)

print("▶️ Tabela pivotada com user_id x meses:")
df_pivot.show(truncate=False)

▶️ Tabela pivotada com user_id x meses:
+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|yt_user          |2023-04|2023-05|2023-06|2023-07|2023-08|2023-09|2023-10|2023-11|2023-12|2024-01|2024-02|2024-03|
+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|tedx Talks       |401    |1028   |1136   |840    |767    |680    |702    |846    |673    |683    |585    |522    |
|CanalKondZilla   |26     |64     |27     |26     |21     |20     |23     |22     |21     |17     |25     |37     |
|LuisitoComunica  |5      |12     |11     |9      |9      |18     |12     |8      |9      |6      |15     |10     |
|checkgate        |8      |22     |26     |28     |23     |14     |13     |17     |14     |17     |22     |29     |
|MoreZoella       |1      |0      |0      |5      |30     |2      |3      |0      |1      |0      |0      |0      |
|luccasneto       |27     |56   